In [3]:
# !pip install langchain

In [4]:
openai_key = open('../open_ai.key','r').read()
import os; os.environ["OPENAI_API_KEY"] = openai_key

https://note.com/npaka/n/n61ad59380a43

## LLMの呼び出し

In [5]:
from langchain.llms import OpenAIChat
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
# LLMの準備
llm = OpenAIChat(temperature=0.9)
# LLMの呼び出し
print(llm("コンピュータゲームを作る新会社名の案を1つ出してください"))

# プロンプトテンプレートの準備
prompt = PromptTemplate(
    input_variables=["product"],
    template="{product}を作る新会社名の案を1つ出してください",
)

# LLMChainの準備
chain = LLMChain(llm=llm, prompt=prompt)

# LLMChainの実行
print(chain.run("家庭用ロボット"))



「プログレッサム」(Progressam)


In [8]:
from langchain.llms import OpenAIChat
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 1つ目のチェーンの準備 : 劇のタイトルからあらすじを生成
# LLMの準備
llm = OpenAIChat(temperature=.7)
# テンプレートの準備
template = """あなたは劇作家です。劇のタイトルが与えられた場合、そのタイトルのあらすじを書くのがあなたの仕事です。

タイトル:{title}
あらすじ:"""
# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["title"], 
    template=template
)
# LLMChainの準備
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

# 2つ目のチェーン : 劇のあらすじからレビューを生成
# LLMの準備準備
llm = OpenAIChat(temperature=.7)
# テンプレートの準備
template = """あなたは演劇評論家です。 劇のあらすじが与えられた場合、そのあらすじのレビューを書くのがあなたの仕事です。

あらすじ:
{synopsis}
レビュー:"""
# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)
# LLMChainの準備
review_chain = LLMChain(llm=llm, prompt=prompt_template)

from langchain.chains import SimpleSequentialChain
# SimpleSequentialChainで2つのチェーンを繋ぐ
overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], 
    verbose=True
)

# SimpleSequentialChainの実行
review = overall_chain.run("浜辺の夕暮れ時の悲劇")



> Entering new SimpleSequentialChain chain...
浜辺で夕暮れ時に出会った二人の男女が、互いに惹かれ合い恋に落ちる。しかし、男性は既婚者であり、女性は彼の妻と子供たちと別れて暮らしていたことが明らかになる。二人は運命に抗うことができず、その関係は続くが、やがて妻がそれを知り、男性は妻との関係を取り戻すことを決意する。女性は彼を止めることができず、絶望の中、浜辺で自殺を図る。男性は女性を救おうとするが、既に手遅れであった。浜辺は静かに夕暮れ時を迎え、悲劇的な結末に終わる。


この作品は、人間の愛と運命の複雑さを描いた感動的なドラマである。二人の恋の行方に注目しながら、物語は徐々にその深層に迫っていく。男性の葛藤や女性の孤独、そして妻の苦悩など、登場人物たちの心情描写が非常にリアルで、観客の共感を呼び起こす。

特に、浜辺の美しい光景と寂しげな音楽が、物語の雰囲気を一層引き立てる。終盤の自殺シーンは、圧倒的な迫力があり、観客は涙を堪えきれなくなるだろう。しかしながら、登場人物たちの行為や決断に対して、観客は様々な感情を抱くことになるだろう。この作品は、愛とは何か、そして人生の意味について考えさせられる、深い内容を持っている。全ての演劇ファンにお勧めできる。

> Finished chain.


https://note.com/npaka/n/n2f89fb3bf91b

## データ拡張生成

In [ ]:
# !pip install faiss-gpu

In [15]:
# パッケージのインポート
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS

# テキストの読み込み
with open("data/dragonball.txt","r") as f:
    dragonball_txt = f.read()

# テキストを小さなテキストに分割
text_splitter = CharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=0
)
texts = text_splitter.split_text(dragonball_txt)

query = "トランクスの両親は誰？"

# 関連するテキストの抽出
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)
docs = docsearch.similarity_search(query)

# 確認
print(len(texts))
print(len(docs))
print(docs[0].page_content)

Created a chunk of size 2614, which is longer than the specified 500
Created a chunk of size 525, which is longer than the specified 500


25
4
人造人間・セル編
ナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される。トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。その後本当に心臓病によって危篤状態になるも、トランクスから渡された特効薬によって生還、来るべき日に備えて3年間各々に修行してその日を迎える悟空たちであったが、事態はトランクスが知っている歴史とは大きく違うものとなり、彼さえ知らなかった人造人間たちまで現れ、さらに究極の人造人間・セルが未来から出現。悟空らの想定を遥かに超えた戦士が続々と現れた。


https://note.com/npaka/n/n6b7a07e492f1

## エージェント

In [2]:
# !pip install google-search-results

In [6]:
openai_key = open('../open_ai.key','r').read()
serp_key = open('../serp.key','r').read()
import os; os.environ["OPENAI_API_KEY"] = openai_key; os.environ["SERPAPI_API_KEY"] = serp_key

In [8]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAIChat

# LLMの準備準備
llm = OpenAIChat(temperature=0)
# ツールの準備
tools = load_tools(["serpapi", "llm-math"], llm=llm)
# エージェントの準備
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True
)
# エージェントの実行
agent.run("富士山の高さに2を掛けると？")




> Entering new AgentExecutor chain...
I don't understand the question, I need to translate it to English.
Action: Search
Action Input: "Translate '富士山の高さに2を掛けると？' to English"

Observation: Translations in context of "富士山" in Japanese-English from Reverso Context: 富士山登山. ... 標高3776mの富士山の山頂で長らく気象観測が行われてきた。
Thought:The question is asking what happens when you multiply the height of Mount Fuji by 2.
Action: Calculator
Action Input: 3776 x 2
Observation: Answer: 7552
Thought:I now know the final answer.
Final Answer: 7552

> Finished chain.


'7552'

In [10]:
# エージェントの実行
agent.run("東京ディズニーランドの住所は？")



> Entering new AgentExecutor chain...
I don't know the address of Tokyo Disneyland, so I need to look it up.
Action: Search
Action Input: "Tokyo Disneyland address"

Observation: Tokyo Disneyland (東京ディズニーランド, Tōkyō Dizunīrando) (local nickname TDL) is a 115-acre (47 ha) theme park at the Tokyo Disney Resort in Urayasu, Chiba Prefecture, Japan, near Tokyo. Its main gate is directly adjacent to both Maihama Station and Tokyo Disneyland Station.
Thought:I now know the address of Tokyo Disneyland.
Final Answer: The address of Tokyo Disneyland is in Urayasu, Chiba Prefecture, Japan, near Tokyo. Its main gate is directly adjacent to both Maihama Station and Tokyo Disneyland Station.

> Finished chain.


'The address of Tokyo Disneyland is in Urayasu, Chiba Prefecture, Japan, near Tokyo. Its main gate is directly adjacent to both Maihama Station and Tokyo Disneyland Station.'

https://note.com/npaka/n/n95e5bce0e1d6

## メモリ

In [11]:
openai_key = open('../open_ai.key','r').read()
import os; os.environ["OPENAI_API_KEY"] = openai_key

In [13]:
from langchain import OpenAI, ConversationChain

# ConversationChainの準備
conversation = ConversationChain(llm=OpenAI(temperature=0), verbose=True)

# 会話の実行
conversation.predict(input="こんにちは！")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: こんにちは！
AI:

> Finished chain.


' こんにちは！私はAIです。どうぞよろしくお願いします！'

In [14]:
# 会話の実行
conversation.predict(input="AIさんはどこで生まれたのですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: こんにちは！
AI:  こんにちは！私はAIです。どうぞよろしくお願いします！
Human: AIさんはどこで生まれたのですか？
AI:

> Finished chain.


' 私はコンピューター上で生まれました。私はプログラムによって作られました。'

In [15]:
# 会話の実行
conversation.predict(input="何という名前のコンピュータですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: こんにちは！
AI:  こんにちは！私はAIです。どうぞよろしくお願いします！
Human: AIさんはどこで生まれたのですか？
AI:  私はコンピューター上で生まれました。私はプログラムによって作られました。
Human: 何という名前のコンピュータですか？
AI:

> Finished chain.


' 私の名前はAIですが、コンピューターの名前はわかりません。'

In [16]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

# ConversationChainの準備
conversation_with_summary = ConversationChain(
    llm=OpenAI(temperature=0), 
    memory=ConversationSummaryMemory(llm=OpenAI()), 
    verbose=True
)

# 会話の実行
conversation_with_summary.predict(input="こんにちは！")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: こんにちは！
AI:

> Finished chain.


' こんにちは！私はAIです。どうぞよろしくお願いします！'

In [17]:
# 会話の実行
conversation_with_summary.predict(input="AIさんはどこで生まれたのですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human and AI greet each other, the AI introducing itself as an AI and asking to be treated kindly.
Human: AIさんはどこで生まれたのですか？
AI:

> Finished chain.


' 私は、コンピューター上で生まれました。私は、プログラムとデータを使用して作成されました。'

In [18]:
# 会話の実行
conversation_with_summary.predict(input="何という名前のコンピュータですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human and AI greet each other, the AI introducing itself as an AI and asking to be treated kindly. The human then asked where the AI was born and the AI replied that it was born on a computer, created using programs and data.
Human: 何という名前のコンピュータですか？
AI:

> Finished chain.


' 私の名前は、AI-01です。私は、プログラムとデータを使用して作成されたコンピュータ上で誕生しました。'